In [1]:
import os
import pandas as pd
%matplotlib inline
import nltk
nltk.download('vader_lexicon')
from datetime import datetime, timedelta,date
from pandas import DataFrame

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/anna/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

Use news api to get all news articles and headline sentiment related to Covid-19 or Coronavirus

In [3]:
# Read api key environment variable
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("newsapikey")

In [4]:
import newsapi

In [5]:
# Create a newsapi client
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=api_key)


In [6]:
# Set start and end datetimes of for 1 days of news
from datetime import datetime, timedelta
end_date = datetime.now()
start_date = end_date + timedelta(-1)
end_date=end_date.strftime("%Y-%m-%d")
start_date=start_date.strftime("%Y-%m-%d")


In [7]:
# Fetch the Covid19 news articles
covid19_news = newsapi.get_everything(
    from_param=start_date,
    to=end_date,
    q="Covid 19",
    language="en",
    page_size=100,
    sort_by="relevancy"
)
# Print number articles found for reference
print(f"Total articles related to Covid-19: {covid19_news['totalResults']}")

Total articles related to Covid-19: 20729


In [8]:
# Create the Covid-19 sentiment scores DataFrame
#covid-19 sentiment list to dataframe
covid19_sentiment_list=[]
for article in covid19_news["articles"]:        
   try: 
        text = article["content"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        scores={"Compound":compound, "Negative":neg, "Neutral":neu, "Positive":pos, "text":text}
        covid19_sentiment_list.append(scores)
        
   except: 
        pass

covid19_sentiment_df=pd.DataFrame(covid19_sentiment_list)
covid19_sentiment_df.head()

,Compound,Negative,Neutral,Positive,text
0,-0.2924,0.056,0.944,0.000,"Airbus is, of course, one of the world’s two m..."
1,0.5859,0.040,0.794,0.167,"In 2019, the global fertility services industr..."
2,-0.0258,0.070,0.862,0.068,It isn’t clear exactly how many employees will...
3,0.3400,0.000,0.911,0.089,There are some wearables out there in the worl...
4,-0.4993,0.158,0.788,0.054,It doesn't help matters for any budding hurric...


In [9]:
# Describe the  Sentiment Related to Covid19/Coronavirus
sentiment_df=covid19_sentiment_df.describe()
sentiment_df

,Compound,Negative,Neutral,Positive
count,97.000000,97.000000,97.000000,97.000000
mean,0.157355,0.030608,0.897979,0.071412
std,0.426943,0.048929,0.084618,0.077924
min,-0.765000,0.000000,0.632000,0.000000
25%,0.000000,0.000000,0.856000,0.000000
50%,0.000000,0.000000,0.892000,0.058000
75%,0.531900,0.067000,1.000000,0.124000
max,0.868900,0.175000,1.000000,0.263000


In [20]:

daily_signal_dict={}
daily_sentiment_dict={}

for i in range(30):

    # Set start and end datetimes of for 1 days of news
    end_date = datetime.now()
    start_date = end_date + timedelta(-i)
    end_date=end_date.strftime("%Y-%m-%d")
    start_date=start_date.strftime("%Y-%m-%d")
    

    # Fetch the Covid19 news articles
    covid19_news = newsapi.get_everything(
    from_param=start_date,
    to=end_date,
    q="Covid 19",
    language="en",
    page_size=100,
    sort_by="relevancy"
    )
    

    #covid-19 sentiment list to dataframe
    covid19_sentiment_list=[]
    for article in covid19_news["articles"]:        
        try: 
            text = article["content"]
            sentiment = analyzer.polarity_scores(text)
            compound = sentiment["compound"]
            pos = sentiment["pos"]
            neu = sentiment["neu"]
            neg = sentiment["neg"]
            scores={"Compound":compound, "Negative":neg, "Neutral":neu, "Positive":pos, "text":text}
            covid19_sentiment_list.append(scores)
        
        except: 
            pass

    covid19_sentiment_df=pd.DataFrame(covid19_sentiment_list)
    

    # Describe the  Sentiment Related to Covid19/Coronavirus
    sentiment_df=covid19_sentiment_df.describe()


    #determine trading signal value based on covid news sentiment 
    if sentiment_df['Neutral'][1] > (sentiment_df['Positive'][1] and sentiment_df['Negative'][1]):
        sentiment_signal=0.0
        sentiment='Neutral'
    elif sentiment_df['Negative'][1] > (sentiment_df['Positive'][1] and sentiment_df['Neutral'][1]):
        sentiment_signal=-1.0
        sentiment='Negative'
    elif sentiment_df['Positive'][1] > (sentiment_df['Negative'][1] and sentiment_df['Netural'][1]):
        sentiment_signal=1.0
        sentiment='Positive'

    daily_signal_dict.update({start_date:sentiment_signal})
    daily_sentiment_dict.update({start_date:sentiment})
   
#convert dictionary to dataframe     
daily_signal_df=DataFrame.from_dict(daily_signal_dict,orient='index',columns=['Signal'])
daily_sentiment_df=DataFrame.from_dict(daily_sentiment_dict,orient='index',columns=['Sentiment'])

sentiment_signal_df=pd.concat([daily_signal_df,daily_sentiment_df],join='inner', axis=1)


In [21]:
sentiment_signal_df

,Signal,Sentiment
2020-06-25,0.0,Neutral
2020-06-24,0.0,Neutral
2020-06-23,0.0,Neutral
2020-06-22,0.0,Neutral
2020-06-21,0.0,Neutral
2020-06-20,0.0,Neutral
2020-06-19,0.0,Neutral
2020-06-18,0.0,Neutral
2020-06-17,0.0,Neutral
2020-06-16,0.0,Neutral
